# Занятие 12 
# Прикладная алгебра и численные методы
## Теорема Фробениуса-Перрона, Pagerank

In [ ]:
import numpy as np
from numpy import random
import sympy
from sympy import latex, Matrix
import scipy.linalg
import numpy.linalg
from IPython.display import Latex

In [ ]:
sympy.__version__, scipy.__version__, np.__version__

('1.11.1', '1.10.1', '1.22.4')

## Сравнение матриц
$A \ge B$, $a_{ij} \ge b_{ij}$ $\forall i, j$

$A > B$, $a_{ij} > b_{ij}$ $\forall i, j$
## Неотрицательная матрица
$A \ge 0$, $a_{ij} \ge 0$ $\forall i, j$
## Положительная матрица
$A > 0$, $a_{ij} \ge 0$ $\forall i, j$


## Теорема Перрона 1907
Пусть $A$ - квадратная матрица $n\times n$, $A > 0$.

Тогда 

(а) Существует положительное собственное значение $\hat\lambda = \hat\lambda_A > 0$ такое, что для любого собственного значения $\lambda_i = \lambda_i(A)$ выполняется $\hat\lambda_A \ge|\lambda_i|$, т.е. $\hat\lambda_A = \rho(A)$.

(б) Существует (и единственный с точностью до умножения на число) собственный вектор $\hat x_A > 0$, соответствующий собственному значению  $\hat\lambda_A$.

## Пример 1.

Построим случайную квадратную матрицу размерности 10, найдем ее спектр, спектральный радиус и собственный вектор, соответствующий максимальному собственному значению.

In [ ]:
n = 10
A1 = np.array([[random.randint(1, 10) for i in range(n)] for j in range(n)])
spectrA1, eigvectors = np.linalg.eig(A1)
spectrA1 = [(round(item.real) + 1j * round(item.imag)) for item in spectrA1]
display(Latex(f"""A1 = {latex(Matrix(A1))}\\\\
spectrA1 = {spectrA1}\\\\
eigvectors = {latex(Matrix(eigvectors.real.round(2)))}"""))

<IPython.core.display.Latex object>

Проверим, что первое собственное значение - наибольшее по модулю:

In [ ]:
lambda_max = abs(spectrA1[0])
print(f'Первое с.з {spectrA1[0]} максимальное по модулю, т.е.')
for i, item in enumerate(spectrA1[1:]): 
  print(f'больше модуля с.з. {item} номер {i + 2}', lambda_max >= abs(item))

Первое с.з (48+0j) максимальное по модулю, т.е.
больше модуля с.з. (-2+7j) номер 2 True
больше модуля с.з. (-2-7j) номер 3 True
больше модуля с.з. (3+0j) номер 4 True
больше модуля с.з. (1+0j) номер 5 True
больше модуля с.з. (-6+4j) номер 6 True
больше модуля с.з. (-6-4j) номер 7 True
больше модуля с.з. (-3+2j) номер 8 True
больше модуля с.з. (-3-2j) номер 9 True
больше модуля с.з. (-6+0j) номер 10 True


Выведем на экран собственный вектор $\hat x_A$, соответствующий максимальному собственному значению.

Проверим, что с.в. соответствует с.з., используем np.allclose для сравнения.


In [ ]:
x_lambda_max = eigvectors[:, 0]
print(f"""Максимальное с.з. {lambda_max},\nс.в.\n{x_lambda_max.real.round(2)},
A1 x_lambda_max = lambda_max x_lambda_max - {np.allclose(A1 @ x_lambda_max, lambda_max*x_lambda_max)}""")

Максимальное с.з. 48.0,
с.в.
[0.4  0.21 0.35 0.35 0.27 0.26 0.22 0.38 0.3  0.36],
A1 x_lambda_max = lambda_max x_lambda_max - False


Разберемся, почему получилось False. Выведем на экран левую и правую части равенства $A_1 x = \lambda x$:

In [ ]:
print(f'A1 @ x_lambda_max\n{np.round(A1 @ x_lambda_max, 1)}\n\
lambda_max*x_lambda_max\n{np.round(lambda_max*x_lambda_max, 1)}')

A1 @ x_lambda_max
[19.2+0.j 10.1+0.j 16.7+0.j 17. +0.j 13. +0.j 12.3+0.j 10.8+0.j 18.3+0.j
 14.3+0.j 17.6+0.j]
lambda_max*x_lambda_max
[19.1+0.j 10.1+0.j 16.6+0.j 17. +0.j 12.9+0.j 12.3+0.j 10.7+0.j 18.2+0.j
 14.2+0.j 17.5+0.j]


Видим, что дело в недостаточной точности вычисления с.з. и с.в.

Проведем сравнение с более низкой точностью, для этого изменим необязательный параметр atol функции np.allclose:

In [ ]:
print(f'A1 x_lambda_max = lambda_max x_lambda_max - \
{np.allclose(A1 @ x_lambda_max, lambda_max*x_lambda_max, atol=0.2)}')

A1 x_lambda_max = lambda_max x_lambda_max - True


У np.allclose есть еще один необязательный параметр rtol, он задает относительную погрешность (atol соответствует абсолютной погрешности). При сравнении оба параметра использовуются одновременно. 

По умолчанию rtol=1e-05, atol=1e-08.

https://numpy.org/doc/stable/reference/generated/numpy.allclose.html

In [ ]:
print(f'A1 x_lambda_max = lambda_max x_lambda_max - \
{np.allclose(A1 @ x_lambda_max, lambda_max*x_lambda_max, rtol=0.01)}')

A1 x_lambda_max = lambda_max x_lambda_max - True


Проверка приближенного равенства с заданной точностью осуществляется внутри np.allclose с помощью функции within_tol

def within_tol(x, y, atol, rtol):

$\quad$  with errstate(invalid='ignore'):

$\quad$   return less_equal(abs(x-y), atol + rtol * abs(y))


within_tol проверяет выполнение условия $|x - y| \le atol + rtol\cdot|y|$

Cравним такие два вектора-строки:

In [ ]:
BB1 = np.array([0.5, 1.9, 1000.1, 100000.9])
BB2 = np.array([0.6, 1.9, 1000.1, 100000.9])
BB3 = np.array([0.499, 1.9, 1000.21, 100000.1])
for pair in (('BB1 == BB2', BB1, BB2), ('BB1 == BB3', BB1, BB3)):
  my_text, B1, B2 = pair
  print(f"""{my_text}\natol = 0.1 {np.allclose(B1, B2, atol=0.1)}\n\
rtol = 0.01 {np.allclose(B1, B2, rtol=0.01)}
atol = 0.1,  rtol = 0.01 {np.allclose(B1, B2, atol=0.1, rtol=0.01)}""")

BB1 == BB2
atol = 0.1 True
rtol = 0.01 False
atol = 0.1,  rtol = 0.01 True
BB1 == BB3
atol = 0.1 True
rtol = 0.01 True
atol = 0.1,  rtol = 0.01 True


Обратите внимание, что разность двух последних элементов сравниваемых строк $BB_1$ и  $BB_3$  больше, чем 0.1, но при atol = 0.1 np.allclose возвращает True, это проявляется значение по умолчанию rtol=1e-05.

Если требуется учитывать только абсолютное отклонение, то нужно установить rtol=0.

In [ ]:
for pair in (('BB1 == BB2', BB1, BB2), ('BB1 == BB3', BB1, BB3)):
  my_text, B1, B2 = pair
  print(f"""{my_text}\natol = 0.1 {np.allclose(B1, B2, atol=0.1, rtol=0)}""")

BB1 == BB2
atol = 0.1 True
BB1 == BB3
atol = 0.1 False


# Важно!!!
По тереме Фробениуса-Перрона собственный вектор, соответствующий максимальному собственному значению ПОЛОЖИТЕЛЬНЫЙ, т.е. все его элементы положительные, а у нас не так, проверим:

In [ ]:
print('С.в. x_lambda_max, соответствующий корню Перрона A1, положительный?',
      x_lambda_max > 0)

С.в. x_lambda_max, соответствующий корню Перрона A1, положительный? [ True  True  True  True  True  True  True  True  True  True]


Поскольку собственные векторы определяются с точностью до множителя, который еще и может быть отрицательным, то имеет смысл проверять не положительность всех координат, а одного ли все они знака.

## Неразложимая матрица
Матрица $A_{n\times n} \ge 0$ называется неразложимой, если одновременной перестановкой строк и столбцов ее нельзя привести к виду
$$
A = 
\left(
\begin{matrix}
  & P & &| & Q &\\
- & - & &| & - & - \\
0 & 0 & 0 &|  &  &  \\
0 & 0 & 0 &|  & R &  \\
0 & 0 & 0 &|  &  &  
\end{matrix}
  \right)
$$
где $P$ и $R$ квадратные матрицы размерности $n > 0$.
## Свойство 1. 
Матрица смежности $A$ взвешенного ориентированного графа $G$ неразложимая тогда и только тогда, когда граф $G$ сильно связный, т.е. в нем есть путь из любой вершины в любую.
## Предложение 1.
Матрица $A \ge 0$ неразложимая тогда и только тогда, когда для любых $i$ и $j$ существует $k < n$ такое что
$$
(A^k)_{ij} > 0.
$$
## Теорема Перрона-Фробениуса (Фробениус, 1913)
Если $A \ge 0$ неразложимая, то выполняется теорема Перрона.
## Следствие 1.
Если $A \ge 0$, то существует $\hat \lambda \ge 0$ - вещественное собственное значение, равное спектральному радиусу $\rho(A)$ и такое, что ему соответствует собственный вектор $\hat x \ge 0$ (возможно, не единственный!)

## Пример 2
Покажем что для матрицы $A_2$ выполняется теорема Перрона
$$
A_2 = 
\left(
\begin{matrix}
0 & 3 & 0 & 5 & 9\\
3 & 7 & 4 & 0 & 0\\
0 & 4 & 0 & 0 & 0\\
5 & 0 & 0 & 0 & 2\\
9 & 6 & 0 & 2 & 8  
\end{matrix}
  \right)
$$

In [ ]:
A2 = np.array([[0, 3, 0, 5, 9],
               [3, 7, 4, 0, 0],
               [0, 4, 0, 0, 0],
               [5, 0, 0, 0, 2],
               [9, 6, 0, 2, 8]])
spectrA2, eigvectorsA2 = np.linalg.eig(A2)
spectrA2 = [(round(item.real) + 1j * round(item.imag)) for item in spectrA2]
display(Latex(f"""A2\n{latex(Matrix(A2))}\\\\
spectrA1 = {spectrA2},\\\\
eigvectors = {latex(Matrix(eigvectorsA2.real.round(2)))}"""))

<IPython.core.display.Latex object>

Как видим, максимальное собственное значение вещественное и положительное, а его собственный вектор - отрицательный. Умножим собственный вектор на -1 и проверим выполнение равенства $A_2 x = \lambda x$:

In [ ]:
lambda_maxA2 = spectrA2[0]
x_lambda_maxA2 = - eigvectorsA2[:, 0]
print(f"""Максимальное с.з. {lambda_maxA2},\nс.в.{x_lambda_maxA2.real.round(2)},
A2 x_lambda_max = lambda_max x_lambda_max - 
{np.allclose(A2 @ x_lambda_maxA2,
             lambda_maxA2*x_lambda_maxA2,
             atol = 0.1, rtol = 0.1)}""")

Максимальное с.з. (16+0j),
с.в.[0.54 0.19 0.05 0.26 0.78],
A2 x_lambda_max = lambda_max x_lambda_max - 
True


## Алгоритм PageRank
Пусть в сети $n$ страниц, присвоим им номера 1, ..., $n$.

Обозначим $a_{ij}$ число ссылкок станицы $j$ на страницу $i$.
$$
\bar x = 
\left(
\begin{matrix}
p_1\\p_2\\...\\p_i\\...\\p_n
\end{matrix}
\right)  
$$
$p_i$ - вероятность того, что после большого числа случайных переходов со страницы на страницу пользователь попадет на страницу $i$.

Пусть $p_{ij}$ - вероятность перехода со страницы $j$ на страницу $i$:
$$
p_{ij} = \frac{a_{ij}}{\sum_{k}a_{kj}}
$$
Обозначим $P = (p_{ij})$ - матрица вероятностей переходов.

Если 
$$
x^t = 
\left(
\begin{matrix}
x_1^t\\x_2^t\\...\\x_i^t\\...\\x_n^t
\end{matrix}
\right),  
$$
$x_i^t$ - вероятность оказаться на странице $i$ после $t$ шагов,
то 
$$
x_i^{t + 1} = \sum_{j}x_j^t p_{ij} = P_i x^t,
$$
т.е.
$$
x^{t + 1} = P x^t
$$
При $t\to\infty$ получим СЛАУ $x = Px$, т.е. вектор $x$ - собственный вектор матрицы вероятностей переходов, соответствующий собственному значению 1.
## Уточненная модель.
Пользователь выбирает случайную страницу с вероятностью $\alpha$, переходит по случайной ссылке с вероятностью $\beta = 1 - \alpha$.
В этой модели
$$
x_i^{t + 1} = \beta P_i x^t + \alpha\frac{1}{n},
$$
т.е.
$$
x^{t + 1} = (1 - \alpha)P x^t + \frac{1}{n}\alpha
\left(
\begin{matrix}
1\\...\\1
\end{matrix}
\right)
= \left[(1 - \alpha)P  + \frac{\alpha}{n}\left(
\begin{matrix}
1 & ... & 1\\& ...&\\1 & ... & 1
\end{matrix}
\right)\right]x^t = \tilde P x^t
$$
$||\tilde P||_1 = \max_{j} |\tilde P|_1 = \max \{1\} = 1$, т.е. $|\lambda_\max| = \rho(\tilde P) \le 1$.

Итак, $\lambda_1 = 1 \ge |\lambda_\max|$, т.е. $\lambda_1 =\hat\lambda$.
Значит, существует единственный вектор $x = \hat x > 0$. Первоначально значение параметра $\alpha = 0.15$.
## Пример 3.
Найдем самую влиятельную вершину в графе, заданном матрицей смежности
$$
P = \left(
\begin{matrix}
1/6 & 1/3 & 1/3\\1/2 & 1/12 & 1/3\\1/3 & 7/12 & 1/3
\end{matrix}
\right)
$$


In [ ]:
P = np.array([[1 / 6, 1 / 3, 1 / 3],
              [1 / 2, 1 / 12, 1 / 3],
              [1 / 3, 7 / 12, 1 / 3]])
x0 = np.array([[1 / 3], [1 / 3], [1 / 3]])
n = int(input('Введите число итераций\n'))
for i in range(1, n + 1):
  x0 = P @ x0
  print(f'итерация {i}: x{i} = {x0.T[0].real.round(2)}')

Введите число итераций
3
итерация 1: x1 = [0.28 0.31 0.42]
итерация 2: x2 = [0.29 0.3  0.41]
итерация 3: x3 = [0.29 0.31 0.41]


In [ ]:
ev = np.linalg.eig(P)
display(*[item.real.round(2) for item in ev])

array([ 1.  , -0.17, -0.25])

array([[ 0.49,  0.27, -0.  ],
       [ 0.52,  0.53, -0.71],
       [ 0.7 , -0.8 ,  0.71]])

Выделим собственный вектор, соответствующий собственному значению 1:

In [ ]:
ev1  = np.array([ev[1][:, k] for k in range(ev[1].shape[0]) if np.allclose(ev[0][k], 1)])
print(f'Собственному значению 1 соответствует собственный вектор {ev1.real.round(2)}')

Собственному значению 1 соответствует собственный вектор [[0.49 0.52 0.7 ]]


Нормализуем этот вектор по норме $\infty$:

In [ ]:
ev1 = ev1 / np.linalg.norm(ev1, ord=np.inf)
print(f'Нормализованный собственный вектор {ev1.real.round(2)}')

Нормализованный собственный вектор [[0.29 0.3  0.41]]


Как видно из итерационного процесса, именно к этому вектору сходится последовательность.

Вывод - самая влиятельная вершина третья, ей соответствует самая большая предельная вероятность.